In [1]:
from graph import Graph
from pass_manager import pass_m
import cv2
import numpy as np
import array

# load onnx mode

In [2]:
inp_dim = [1, 1, 28, 28]
graph = Graph('mnist', './data/mnist-8.onnx', inp_dim, debug=1)

tensor: Parameter6[8] (8,1,1) producer:  ['_param'] 	consumer:  ['Plus30']
tensor: Input3[784] (1,1,28,28) producer:  [] 	consumer:  ['Convolution28']
tensor: Parameter194[10] (1,10) producer:  ['_param'] 	consumer:  ['Plus214']
tensor: ReLU114_Output_0[1] () producer:  ['ReLU114'] 	consumer:  ['Pooling160']
tensor: Plus214_Output_0[1] () producer:  ['Plus214'] 	consumer:  []
tensor: Plus30_Output_0[1] () producer:  ['Plus30'] 	consumer:  ['ReLU32']
tensor: Times212_Output_0[1] () producer:  ['Times212'] 	consumer:  ['Plus214']
tensor: Parameter5[200] (8,1,5,5) producer:  ['_param'] 	consumer:  ['Convolution28']
tensor: Pooling160_Output_0_reshape0_shape[2] (2) producer:  ['_param'] 	consumer:  ['Times212_reshape0']
tensor: Parameter88[16] (16,1,1) producer:  ['_param'] 	consumer:  ['Plus112']
tensor: Pooling66_Output_0[1] () producer:  ['Pooling66'] 	consumer:  ['Convolution110']
tensor: Parameter193_reshape1_shape[2] (2) producer:  ['_param'] 	consumer:  ['Times212_reshape1']
tensor:

this print all tensors in graph, with tensors' producer nodes and consumer nodes
- the tensor `Input3` with empty producer is the graph's input tensor
- the tensor `Plus214_Output_0` with empty consumer is the graph's output tensor

# print(graph)
this will print all nodes in graph with (node.name, node.op_type)

In [3]:
print(graph)

name:Times212_reshape1	op_type:Reshape
name:Convolution28	op_type:Conv
name:Plus30	op_type:Add
name:ReLU32	op_type:Relu
name:Pooling66	op_type:MaxPool
name:Convolution110	op_type:Conv
name:Plus112	op_type:Add
name:ReLU114	op_type:Relu
name:Pooling160	op_type:MaxPool
name:Times212_reshape0	op_type:Reshape
name:Times212	op_type:MatMul
name:Plus214	op_type:Add



# run pass
this will fuse conv_add, and remove reshape node

In [4]:
graph = pass_m.run_all_pass(graph)
print(graph)

name:Convolution28	op_type:Conv_Add_fused
name:ReLU32	op_type:Relu
name:Pooling66	op_type:MaxPool
name:Convolution110	op_type:Conv_Add_fused
name:ReLU114	op_type:Relu
name:Pooling160	op_type:MaxPool
name:Times212	op_type:MatMul_Add_fused



# dump graph ir

In [5]:
graph.dump_ir()

%0 = Input3
%1 = Parameter5
%2 = Parameter6
%3 = Conv_Add_fused(%0,%1,%2)
%4 = Relu(%3)
%5 = MaxPool(%4)
%6 = Parameter87
%7 = Parameter88
%8 = Conv_Add_fused(%5,%6,%7)
%9 = Relu(%8)
%10 = MaxPool(%9)
%11 = Parameter193_reshape1
%12 = Parameter194
%13 = MatMul_Add_fused(%10,%11,%12)



# inference, get all tensors

In [6]:
graph.infershape()
tensor_map = graph.get_all_tensors(debug=1)

0 tensor: Input3[784] (1,1,28,28)
1 tensor: Parameter5[200] (8,1,5,5)
2 tensor: Parameter6[8] (8)
3 tensor: Plus30_Output_0[6272] (1,8,28,28)
4 tensor: ReLU32_Output_0[6272] (1,8,28,28)
5 tensor: Pooling66_Output_0[1568] (1,8,14,14)
6 tensor: Parameter87[3200] (16,8,5,5)
7 tensor: Parameter88[16] (16)
8 tensor: Plus112_Output_0[3136] (1,16,14,14)
9 tensor: ReLU114_Output_0[3136] (1,16,14,14)
10 tensor: Pooling160_Output_0[256] (1,256)
11 tensor: Parameter193_reshape1[2560] (256,10)
12 tensor: Parameter194[10] (10)
13 tensor: Plus214_Output_0[10] (1,10)


# mnist demo (with preprocess, postprocess)

In [8]:
def preprocess(img_path, inp_dim):
    image = cv2.imread(img_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.resize(gray,(inp_dim[2], inp_dim[3])).astype(np.float32) / 255
    input_data = np.reshape(gray, inp_dim)
    return input_data

def postprocess(out_data):
    prediction = int(np.argmax(np.array(out_data).squeeze(), axis=0))
    print("prediction: ", prediction)

def mnist_demo(image_path):
    inp_data = preprocess(image_path, inp_dim)
    graph.feed_input_data(inp_data, inp_dim)
    graph.run()
    out_data = graph.get_output_data()
    postprocess(out_data)

mnist_demo("data/0.jpg")
mnist_demo("data/3.jpg")
mnist_demo("data/6.jpg")

prediction:  0
prediction:  3
prediction:  6


# dump output data of each node
the dumped data can be found in `data/` directory

In [9]:
inp_data = preprocess("data/6.jpg", inp_dim)
graph.feed_input_data(inp_data, inp_dim)
graph.run(debug=1)
out_data = graph.get_output_data()


dumping output tensor of each nodes
.... data/Convolution28_1_8_28_28.bin
.... data/ReLU32_1_8_28_28.bin
.... data/Pooling66_1_8_14_14.bin
.... data/Convolution110_1_16_14_14.bin
.... data/ReLU114_1_16_14_14.bin
.... data/Pooling160_1_256.bin
.... data/Times212_1_10.bin


# check output data

In [10]:
print(out_data)
postprocess(out_data)

[[ 2.79700494e+00 -1.24417000e+01  2.06825852e-01 -3.55096745e+00
   1.44057125e-02  5.13820505e+00  1.75181866e+01 -1.69534550e+01
   2.51717901e+00 -5.37660265e+00]]
prediction:  6


# save input data


In [ ]:
inp_data = preprocess("data/6.jpg", inp_dim)
import array
data=array.array('f',inp_data.flatten())
with open('data/input_6.bin','wb') as fp:
    data.tofile(fp)